In [ ]:
pip install dask

In [ ]:
pip install dask distributed --upgrade

In [ ]:
pip install wandb

In [ ]:
pip install torch-optimizer

In [ ]:
pip install -U triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir_sm90#subdirectory=python

In [ ]:
pip install -U transformers einops

In [1]:
import torch
import transformers
import logging
import shutil
import os
import torch
from torch.utils.data import Dataset, DataLoader

# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a file handler
handler = logging.FileHandler('training.log')
handler.setLevel(logging.INFO)

# Create a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add the handlers to the logger
logger.addHandler(handler)
logger.addHandler(console_handler)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available() )
if not torch.cuda.is_available():
    raise "Must have CUDA!"

True


In [2]:
!nvidia-smi

Mon Jul 17 01:08:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 PCIe    On   | 00000000:06:00.0 Off |                    0 |
| N/A   45C    P0    51W / 350W |      3MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
print("starting...")

import os
import json
import random
import numpy as np
import transformers
import wandb
import time
import pickle
from multiprocessing import Pool
from typing import List
import time
print("loaded libs...")
os.environ["TOKENIZERS_PARALLELISM"] = "true"

from transformers import PreTrainedTokenizerFast
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
class JsonDataset(Dataset): 
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            self.data = json.load(file)
        
    def __len__(self):
            return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

print("starting for real")
logger.info("start generation of dataset")
# Instantiate the dataset
start_time = time.time()  # Start timing
tokenizer.pad_token = tokenizer.eos_token
dataset = JsonDataset('dotnet_csharp_instruct_gpl-xl.json')

def collate_fn_minimal_padding(batch, max_length=2000):
    texts = [item for item in batch]
    
    # Tokenize the batch with padding
    encoded_batch = tokenizer.batch_encode_plus(
        texts,
        truncation=True,
        max_length=max_length,
        padding='longest',  # Pad to the longest sequence in the batch
        return_tensors='pt',
        return_attention_mask=False
    )
    
    return encoded_batch

end_time = time.time()  # End timing
logger.info(f'dataset initialised in {end_time-start_time}s')
i = 0

starting...
loaded libs...


2023-07-17 01:08:25,475 - __main__ - INFO - start generation of dataset


starting for real


2023-07-17 01:08:34,708 - __main__ - INFO - dataset initialised in 9.229364395141602s


In [4]:
import shutil
from glob import glob

name =  "mosaicml/mpt-7b-instruct" #'mosaicml/mpt-7b-instruct'

# Clear Cache
#cache_dir = os.getenv("TRANSFORMERS_CACHE", os.path.join(os.getenv("HOME"), ".cache", "huggingface", "transformers"))
#shutil.rmtree(cache_dir, ignore_errors=True)

# Load Config and Modify it
config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.max_seq_len = 2048 #4096
config.attn_config['attn_impl'] ='triton' # 'torch' #'triton'#
config.init_device = 'cuda:0' 

# Enable logging for transformers library
transformers.logging.set_verbosity_info()

# Load Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    name,
    config=config,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
# Get list of all .pt files in your directory
all_files = glob('data/snapshot*.pt')

# Fetch the latest file
latest_file = max(all_files, key=os.path.getctime)

# Load weights from the latest snapshot
model.load_state_dict(torch.load(latest_file), strict=False)  # use strict=False if models are not identical


# Save Model to specified path
#model.save_pretrained("./data/mpt-7b-instruct")


loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--mosaicml--mpt-7b-instruct/snapshots/1fc4634127ec64a45716003578b9cfae23265849/pytorch_model.bin.index.json
Instantiating MPTForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "_from_model_config": true,
  "transformers_version": "4.30.2",
  "use_cache": false
}



Instantiating an MPTForCausalLM model from /home/ubuntu/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/1fc4634127ec64a45716003578b9cfae23265849/modeling_mpt.py
You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MPTForCausalLM.

All the weights of MPTForCausalLM were initialized from the model checkpoint at mosaicml/mpt-7b-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MPTForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--mosaicml--mpt-7b-instruct/snapshots/1fc4634127ec64a45716003578b9cfae23265849/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "transformers_version": "4.30.2",
  "use_cache": false
}



<All keys matched successfully>

In [5]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Modelsize: {model_size/1000**2:.1f}M parameters")


Modelsize: 6649.3M parameters


In [6]:
!nvidia-smi

Mon Jul 17 01:08:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 PCIe    On   | 00000000:06:00.0 Off |                    0 |
| N/A   43C    P0    80W / 350W |  26385MiB / 81559MiB |     12%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import threading
import queue
from torch.optim import AdamW
# Wandb initialization


logger.info("getting ready to train")

model.train()

def set_trainable_layers(model, k=0):
    # Get all layers
    layers = [module for module in model.modules() if list(module.parameters())]
        
    # Freeze all parameters first
    for layer in layers:
        for param in layer.parameters():
            param.requires_grad = False
    
    logger.info(f"totall layers in model { len(layers)}")
    # Then unfreeze the parameters of the top `n_trainable_layers`
    num_layers = len(layers)
    top_layers = 5
    summ_trainable_params = 0
    summ_trainable_layers = 0
    for i, layer in enumerate(layers):
        if  (i % (18+k) == 0 and i > 10) or i >= num_layers - top_layers:
            summ_trainable_layers += 1
            for param in layer.parameters():
                param.requires_grad = True
                summ_trainable_params += 1
    logger.info(f"trainable layers { summ_trainable_layers}")
    logger.info(f"trainable tensors {summ_trainable_params}")
            

trainable_params_cnt = set_trainable_layers(model)
lossfct = torch.nn.CrossEntropyLoss()

logger.info("starting training")

end_time = time.time()
acc_dt1 = 0
acc_dt2 = 0
acc_loss = 0
i = 0
for epoch in range(5):  # assuming 10 epochs
    i = 0
    set_trainable_layers(model, k=epoch)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)
    data_loader = DataLoader(dataset, 
                         batch_size=8,
                         shuffle=True, 
                         pin_memory=True,
                         num_workers=16,
                         persistent_workers=True, 
                         collate_fn=collate_fn_minimal_padding)
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        i += 1

        start_time = time.time()  # Start timing
        data_acc_time = start_time-end_time
        acc_dt2 += data_acc_time
        if i< 10 or  i % 1000 == 0:
            logger.info(f'getting new bach time {data_acc_time}s')

        # Model forward pass
        outputs = model(input_ids)

        last_token_output = outputs.logits[0,-1].view(1,-1)
        # Get the predictions and shift the targets
        predictions =  outputs.logits[:, :-1].contiguous().view(-1, outputs.logits.shape[-1])
        shifted_input_ids = input_ids[:, 1:].contiguous().view(-1)

        loss = lossfct(predictions, shifted_input_ids) 
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()

        end_time = time.time()  # End timing

        if i< 10 or i % 1000 == 0:
            logger.info(f'{i} {loss.item() } {end_time-start_time}s')
        acc_dt1 += end_time-start_time
        acc_loss += float(loss.item())

            
        # Logging and saving snapshots
        if i == 1000 or (i>0 and  i % 10000 == 0):
            torch.save(model.state_dict(), os.path.join('data', f'snapshot_xl_{epoch}_{i}_{acc_loss/1000}.pt'))  # saving snapshots
        
        if i % 1000 == 0:
            logger.info({"Loss": acc_loss/1000,
                      "Train time sec": acc_dt1/1000,
                       "Data acc time sec": acc_dt2/1000,
                         "Iter": i
                      })  # logging to wandb
            acc_dt1 = 0
            acc_dt2 = 0
            acc_loss = 0 
    
    torch.save(model.state_dict(), os.path.join('data', f'snapshot_xl_{epoch}_{i}_{acc_loss/(i % 1000)}.pt'))  # saving snapshots
        

logger.info({"Loss": loss.item()})  # logging to wandb
torch.save(model.state_dict(), os.path.join('data', f'snapshot__xl_{i}.pt'))  # saving snapshots
logger.info("Done")
model.eval()


2023-07-17 01:08:52,100 - __main__ - INFO - getting ready to train
2023-07-17 01:08:52,126 - __main__ - INFO - totall layers in model 293
2023-07-17 01:08:52,128 - __main__ - INFO - trainable layers 20
2023-07-17 01:08:52,129 - __main__ - INFO - trainable tensors 21
2023-07-17 01:08:52,131 - __main__ - INFO - starting training
2023-07-17 01:08:52,153 - __main__ - INFO - totall layers in model 293
2023-07-17 01:08:52,154 - __main__ - INFO - trainable layers 20
2023-07-17 01:08:52,155 - __main__ - INFO - trainable tensors 21
2023-07-17 01:08:52,835 - __main__ - INFO - getting new bach time 0.7023117542266846s
2023-07-17 01:08:57,306 - __main__ - INFO - 1 0.216796875 3.8086397647857666s
2023-07-17 01:08:57,308 - __main__ - INFO - getting new bach time 0.6645102500915527s
2023-07-17 01:08:58,622 - __main__ - INFO - 2 0.22265625 0.6345367431640625s
2023-07-17 01:08:58,624 - __main__ - INFO - getting new bach time 0.6816258430480957s
2023-07-17 01:09:00,191 - __main__ - INFO - 3 0.267578125 

KeyboardInterrupt: 